In [1]:
import pandas as pd

## I guess we're trying to figure out who is stealing fuel
- if fuel is clandestinely entering a fuel tank without a transaction record then...
    - the truck/unit will be traveling more miles than is possible with purchased fuel
    - so, presumably, I should be able to compare a unit's "apparent" calculated mpg with it's nominal mpg
    - units with unrealistically high mpg's would be the receivers of untracked fuel
        - perhaps there are even trucks with unrealistically low mpgs who are unwilling "donors" of untracked fuel
    - So, Do I have records of fuel consumption and fuel miles traveled?

In [3]:
#read in the csvs
events_fuel_df = pd.read_csv('data/Extranet2.ExactFuelEvents.csv')

In [7]:
events_fuel_df.head()

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2


In [13]:
#events_fuel_df.info()

In [4]:
levels_fueltank_df = pd.read_csv('data/Extranet2.ExactFuelTankLevels.csv')

In [9]:
levels_fueltank_df.head()

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.60,12667508
1,10391482,1,72.4,144.80,12667510
2,10391505,1,57.1,114.20,12667533
3,10391507,1,80.9,161.80,12667535
4,10391514,1,56.9,142.25,12667542


In [22]:
units_df = pd.read_csv('data/IBGEFILE.UNITS.csv')
# select UNUNIT, UNYEAR, UNMAKE, UNSER from IBGEFILE.UNITS 
#where (UNTERM = 0 OR UNTERM >= 2017001) 
#and UNUNIT NO LIKE 'T%' and trim (UNUNIT) NOT LIKE '99%'
units_df.columns = ['unit_number', 'model_year', 'make']

In [23]:
units_df.head()

,unit_number,model_year,make
0,535,1996,PETERBILT
1,534,1996,PETERBILT
2,533,1996,PETERBILT
3,532,1996,PETERBILT
4,531,1996,PETERBILT


In [6]:
ffp_df = pd.read_csv('data/BGEIDSC.EF2EFFP.csv')
#

In [11]:
ffp_df.head()

,TRUCK,ACTIVE,SENDFUEL,SENDROUTE,TANKCAP,AVGMPG,SATTYPE
0,1,Y,Y,Y,200,6.0,
1,243,Y,Y,Y,200,6.0,
2,244,Y,Y,Y,200,6.0,
3,245,Y,Y,Y,200,6.0,
4,246,Y,Y,Y,200,6.0,


In [14]:
ffp_df.AVGMPG.value_counts()

6.0    836
Name: AVGMPG, dtype: int64

In [15]:
transaction_df = pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv')
#Actual fuel transactions will have an item code of:
act_fuel_transactions = ['ULSD', 'FUEL', 'CDSL', 'DSL1', 'BDSL']
# and could be in any of the item slots 1-6

In [20]:
transaction_df.columns = ['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
                         'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 
                         'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
                          'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
                          'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6',
                         ]

In [21]:
transaction_df.head()

,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,,0.00,,0.0,,0.0,,0.0,,0
1,42575331,20170101,48,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0
2,42578678,20170101,134,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,...,DEFD,4.12,,0.0,,0.0,,0.0,,0
3,42578722,20170101,137,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0
4,42579695,20170101,302,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0
